In [ ]:
## Import Libraries

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# File management
import os
from tensorflow.keras.utils import image_dataset_from_directory

# Import relevant libraries
import PIL
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Rescaling
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model


import pydot
from IPython.display import SVG

%matplotlib inline

In [ ]:
# important functions
# split the data to improve debug performance
def get_sample_dataset(dataset, percentage):
    size = int(dataset.cardinality().numpy())
    
    # Specify seed to always have the same split distribution between runs
    dataset = dataset.shuffle(size, seed=12)
    
    sample_size = int(percentage*size)
    sample = dataset.take(sample_size)
    return sample

def convert_to_numpy(dataset):
    X = list()
    y = list()
    for image, label in dataset.as_numpy_iterator():
        X.append(image.astype("uint8"))
        y.append(label.astype("uint8")[0])
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

def show_bar_chart(dataset, name):
    outputs = dataset.map(lambda x,y: y)
    labels = np.asarray([y[0] for y in outputs.as_numpy_iterator()])
    labels = [classes[i] for i in labels]
    plt.figure(figsize = (15,8))
    sns.countplot(labels)
    plt.title(f"Frequency of each label({name})")
    plt.show()
    
# split data
# https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438
def get_dataset_partitions_tf(dataset, train_split=0.6, val_split=0.2, test_split=0.2):
    assert (train_split + test_split + val_split) == 1
    
    size = int(dataset.cardinality().numpy())
    
    # Specify seed to always have the same split distribution between runs
    dataset = dataset.shuffle(size, seed=12)
    
    train_size = int(train_split * size)
    val_size   = int(val_split * size)
    
    train_ds = dataset.take(train_size)   
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

def get_img_size(dataset):
    return iter(dataset).get_next()[0].numpy().shape[1:]

def compile_model(model, optimizer, loss):
    model.compile(
        optimizer=optimizer, 
        loss=loss,
        metrics=["accuracy"]
    )

    model.summary()
    
## visualize loss
def show_train_history(hisData,t1,t2): 
    plt.plot(hisData.history[t1])
    plt.plot(hisData.history[t2])
    plt.title('Training History')
    plt.ylabel('value')
    plt.xlabel('epoch')
    plt.legend([t1, t2], loc='upper left')
    plt.show()
    
def get_output_as_numpy(dataset):
    true_y = dataset.map(lambda x,y: y)
    return np.asarray([y[0] for y in true_y.as_numpy_iterator()])

def evaluate(model, history, dataset):
    show_train_history(history, 'loss', 'val_loss')
    show_train_history(history, 'accuracy', 'val_accuracy')
    
    val_x = dataset.map(lambda x,y: x)
    
    true_y = get_output_as_numpy(dataset)
    pred_y = model.predict(dataset)
    
    pred_y = np.argmax(pred_y, axis=1)
    
    # get report
    report = classification_report(
        true_y,
        pred_y,
    )
    
    #get confusion matrix
    cmatrix = confusion_matrix(
        true_y,
        pred_y,
    )

    graph = ConfusionMatrixDisplay(cmatrix)
    graph.plot()
    plt.show()
    print(report)
    
def test_model(model_func, input_size, epochs, batch_sizes, train_ds, val_ds, test_ds, optimizer, loss):
    for batch_size in batch_sizes:
        model = sota_model(input_size)
        compile_model(model, optimizer, loss)
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            shuffle=True,
            batch_size=batch_size,
        )
        print(f"evaluation of the model with batch size: {batch_size}")
        evaluate(model, history, test_ds)

## Obtain the dataset 
Considering the size of the dataset, we will remotely obtain the dataset, from kaggle repository.

[Intel Image Classification](https://www.kaggle.com/datasets/puneet6060/intel-image-classification)

## Get data

In [ ]:
data_dir = "/kaggle/input/intel-image-classification"

train_dir = os.path.join(data_dir, "seg_train", "seg_train")
test_dir = os.path.join(data_dir, "seg_test", "seg_test")

# load variables
label_mode = "int"
img_size = (150, 150) # original size
color_mode='rgb'
batch_size=1

train_ds = image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    label_mode=label_mode,
    color_mode=color_mode,
    image_size=img_size,
    validation_split=0.2,
    seed=123,
    subset="training"
)

val_ds = image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    label_mode=label_mode,
    color_mode=color_mode,
    image_size=img_size,
    validation_split=0.2,
    seed=123,
    subset="validation"
)

test_ds = image_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    label_mode=label_mode,
    color_mode=color_mode,
    image_size=img_size,
    shuffle=True,
)

classes = test_ds.class_names
print(classes)

DEBUG = True
if DEBUG:
    train_ds.shuffle(train_ds.cardinality(),seed=12)
    train_ds = train_ds.take(1200)
    val_ds.shuffle(val_ds.cardinality(),seed=12)
    val_ds = val_ds.take(400)
    test_ds.shuffle(test_ds.cardinality(),seed=12)
    test_ds = test_ds.take(400)

In [ ]:
# normalize image
normalization = Rescaling(1.0/255)
norm_train_ds = train_ds.map(lambda x,y: (normalization(x), y))
norm_val_ds = val_ds.map(lambda x,y: (normalization(x), y))
norm_test_ds = test_ds.map(lambda x,y: (normalization(x), y))

## Visualize data

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10,10))
axes = axes.flatten()
it = iter(train_ds)

for i in range(9):    
    elem = it.get_next()
    image = elem[0].numpy()[0].astype("uint8")
    label = np.argmax(elem[1].numpy())
    
    ax = axes[i]
    ax.imshow(image)
    ax.set_title(classes[label])
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# get frequency for each subset
show_bar_chart(train_ds, "training")
show_bar_chart(val_ds, "validation")
show_bar_chart(test_ds, "test")

### Get input image size

In [ ]:
# get images size
# It is too heavy to keep repeating this instruction
input_size = get_img_size(train_ds)

In [ ]:
# get batch_sizes
batch_sizes = [64, 128, 256]

In [ ]:
# number of epochs
epochs = 25

## Model number 01

In [ ]:
def basic_model01(input_size):
    X_input = Input(input_size, name="input")
    
    # Rescale image to range [0,1]
    X = Rescaling(1.0/255)(X_input)
    
    # Conv01
    X = Conv2D(32, (3, 3), activation='relu', name = 'conv01_32-3')(X)
    X = MaxPooling2D((2, 2), name='max_pool01')(X)
    
    # Conv02
    X = Conv2D(64, (3, 3), activation='relu', name = 'conv02_64-3')(X)
    X = MaxPooling2D((2, 2), name='max_pool02')(X)
    
    # Conv03
    X = Conv2D(128, (3, 3), activation='relu', name = 'conv03_128-3')(X)
    X = MaxPooling2D((2, 2), name='max_pool03')(X)
    
    #Flatten
    X = Flatten()(X)
    
    #FC01
    X = Dense(64, activation='relu', name = 'FC01_64_relu')(X)
    
    #Output
    X = Dense(len(classes), activation="softmax", name = 'softmax_6')(X)
    
    #create
    return Model(inputs = X_input, outputs = X)

## Model 02

In [ ]:
def basic_model02(input_size):
    X_input = Input(input_size)
    
    # Rescale image to range [0,1]
    X = Rescaling(1.0/255)(X_input)
    
    # Conv01
    X = Conv2D(32, (3, 3), activation='relu', name = 'conv01')(X)
    X = MaxPooling2D((2, 2), name='max_pool01')(X)
    
    # Conv02
    X = Conv2D(64, (3, 3), activation='relu', name = 'conv02')(X)
    X = MaxPooling2D((2, 2), name='max_pool02')(X)
    
    # Conv03
    X = Conv2D(128, (3, 3), activation='relu', name = 'conv03')(X)
    X = MaxPooling2D((2, 2), name='max_pool03')(X)

    # Conv04
    X = Conv2D(256, (3, 3), activation='relu', name = 'conv04')(X)
    X = MaxPooling2D((2, 2), name='max_pool04')(X)
    
    #Flatten
    X = Flatten()(X)
    
    #FC01
    X = Dense(64, activation='relu', name = 'FC01')(X)
    
    X = Dropout(0.3)(X)

    #Output
    X = Dense(len(classes), activation="softmax", name = 'output')(X)
    
    #create
    return Model(inputs = X_input, outputs = X)

## Model 03

In [ ]:
def basic_model03(input_size):
    X_input = Input(input_size)
    
    # Rescale image to range [0,1]
    X = Rescaling(1.0/255)(X_input)
    
    # block 01
    X = Conv2D(32, (3, 3), activation='relu', name = 'b1_conv1')(X)
    X = Conv2D(32, (3, 3), activation='relu', name = 'b1_conv2')(X)
    X = MaxPooling2D((2, 2), name='max_pool01')(X)
    
    # Conv02
    X = Conv2D(64, (3, 3), activation='relu', name = 'b2_conv1')(X)
    X = Conv2D(64, (3, 3), activation='relu', name = 'b2_conv2')(X)
    X = MaxPooling2D((2, 2), name='max_pool02')(X)
    
    # Conv03
    X = Conv2D(128, (3, 3), activation='relu', name = 'b3_conv1')(X)
    X = Conv2D(128, (3, 3), activation='relu', name = 'b3_conv2')(X)
    X = MaxPooling2D((2, 2), name='max_pool03')(X)

    # Conv04
    X = Conv2D(256, (3, 3), activation='relu', name = 'b4_conv1')(X)
    X = Conv2D(256, (3, 3), activation='relu', name = 'b4_conv2')(X)
    X = MaxPooling2D((2, 2), name='max_pool04')(X)
    
    #Flatten
    X = Flatten()(X)
    
    #FC01
    X = Dense(128, activation='relu', name = 'FC01')(X)
    
    #FC02
    X = Dense(128, activation='relu', name = 'FC02')(X)
    
    X = Dropout(0.5)(X)

    #Output
    X = Dense(len(classes), activation="softmax", name = 'output')(X)
    
    #create
    return Model(inputs = X_input, outputs = X)

## Model 04

In [ ]:
def basic_model04(input_size):
    X_input = Input(input_size)
    
    # Rescale image to range [0,1]
    X = Rescaling(1.0/255)(X_input)
    
    # block 01
    X = Conv2D(32, (3, 3), name = 'b1_conv1')(X)
    X = BatchNormalization(axis = 3, name = 'b1_bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(32, (3, 3), name = 'b1_conv2')(X)
    X = BatchNormalization(axis = 3, name = 'b1_bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool01')(X)
    
    # Conv02
    X = Conv2D(64, (3, 3), name = 'b2_conv1')(X)
    X = BatchNormalization(axis = 3, name = 'b2_bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), name = 'b2_conv2')(X)
    X = BatchNormalization(axis = 3, name = 'b2_bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool02')(X)
    
    # Conv03
    X = Conv2D(128, (3, 3), name = 'b3_conv1')(X)
    X = BatchNormalization(axis = 3, name = 'b3_bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(128, (3, 3), name = 'b3_conv2')(X)
    X = BatchNormalization(axis = 3, name = 'b3_bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool03')(X)

    # Conv04
    X = Conv2D(256, (3, 3), name = 'b4_conv1')(X)
    X = BatchNormalization(axis = 3, name = 'b4_bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(256, (3, 3), name = 'b4_conv2')(X)
    X = BatchNormalization(axis = 3, name = 'b4_bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool04')(X)
    
    #Flatten
    X = Flatten()(X)
    
    #FC01
    X = Dense(128, activation='relu', name = 'FC01')(X)
    X = Dropout(0.5)(X)
    
    #FC02
    X = Dense(128, activation='relu', name = 'FC02')(X)
    X = Dropout(0.5)(X)

    #Output
    X = Dense(len(classes), activation="softmax", name = 'output')(X)
    
    #create
    return Model(inputs = X_input, outputs = X)

## State-of-the-art Model - VGG16


In [ ]:
def sota_model(input_size):
    
    vgg16 = VGG16(
        input_shape=input_size,
        weights='imagenet',
        include_top=False,
    )
    vgg16.trainable = False
    
    #Flatten
    X = Flatten()(vgg16.output)

    #Output
    X = Dense(len(classes), activation="softmax", name = 'output')(X)
    
    #create
    return Model(inputs = vgg16.input, outputs = X)

## Final model

In [ ]:
def final_model(input_size):
    X_input = Input(input_size)
    
    X = Rescaling(1.0/255)(X_input)

    # Conv01
    X = Conv2D(32, (3, 3), strides=(1,1), padding='same' )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((2, 2))(X)
    
    # Conv02
    X = Conv2D(32, (3, 3), strides=(1,1), padding='same' )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((2, 2))(X)
    
    # Conv03
    X = Conv2D(32, (3, 3), strides=(1,1), padding='same' )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((2, 2))(X)
    
    # Conv04
    X = Conv2D(32, (3, 3), strides=(1,1), padding='same' )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((2, 2))(X)
    
    #Flatten
    X = Flatten()(X)
    
    X = Dense(128, activation='relu')(X)

    X = Dropout(0.2)(X)
    
    #Output
    X = Dense(len(classes), activation="softmax")(X)
    
    #create
    return Model(inputs = X_input, outputs = X)

In [ ]:
loss = "sparse_categorical_crossentropy"
optimizer = "adam"
batch_size = 256

final_model = final_model(input_size)
compile_model(final_model, optimizer, loss)
history = final_model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=batch_size,
    epochs=epochs,
)
evaluate(final_model, history, test_ds)